In [22]:
# Question 8 
import requests
from datetime import datetime, timedelta

def scrap_one_page(page, start_date, end_date):
    # URL de l'API avec les paramÃ¨tres de date
    url = f"https://www.bandsintown.com/choose-dates/fetch-next/upcomingEvents?date=&page={page}&longitude=-74.006&latitude=40.7128&genre_query=all-genres&start_date={start_date}&end_date={end_date}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }


    response = requests.get(url, headers=headers)
    response.raise_for_status()  # VÃ©rifiez si la requÃªte a rÃ©ussi
    data = response.json()  # Essayez de dÃ©coder la rÃ©ponse JSON

    return data

data = scrap_one_page(1, "2024-10-01", "2024-10-31")
data

{'events': [{'artistImageSrc': 'https://photos.bandsintown.com/thumb/18109788.jpeg',
   'properlySizedImageURL': 'https://media.bandsintown.com/110x110/18109788.webp',
   'callToActionRedirectUrl': 'https://www.bandsintown.com/e/105972841-jonathan-arons-official-at-herald-square?came_from=257&utm_medium=web&utm_source=home&utm_campaign=ticket_rsvp',
   'fallbackImageUrl': 'https://assets.prod.bandsintown.com/images/homeIcon/festivalPlaceHolderImage/02.png',
   'artistName': 'Jonathan Arons Official',
   'venueName': 'Herald Square',
   'streamingEvent': False,
   'title': 'Triad Brass for Midtown Music',
   'locationText': 'New York City, NY',
   'pinIconSrc': 'https://assets.prod.bandsintown.com/images/pinIcon.svg',
   'eventUrl': 'https://www.bandsintown.com/e/105972841-jonathan-arons-official-at-herald-square?came_from=257&utm_medium=web&utm_source=home&utm_campaign=event',
   'artistUrl': 'https://www.bandsintown.com/a/15373187-jonathan-arons-official?came_from=257&utm_medium=web&u

In [23]:
# Question 9
import os
import json

def save_json(response, idx_page, folder="data_events"):
    # CrÃ©er le dossier si nÃ©cessaire
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # DÃ©finir le nom du fichier (ex: "page_1.json", "page_2.json", etc.)
    file_name = os.path.join(folder, f"page_{idx_page}.json")
    
    try:
        # Ã‰criture des donnÃ©es dans un fichier JSON
        with open(file_name, 'w', encoding='utf-8') as json_file:
            json.dump(response, json_file, ensure_ascii=False, indent=4)
    except IOError as e:
        print(f"Erreur lors de l'Ã©criture du fichier {file_name}: {e}")

data = scrap_one_page(1, "2024-10-01", "2024-10-31")
save_json(data, 1)

HTTPError: 403 Client Error: Forbidden for url: https://www.bandsintown.com/choose-dates/fetch-next/upcomingEvents?date=&page=13&longitude=-74.006&latitude=40.7128&genre_query=all-genres&start_date=2024-10-3&end_date=2024-10-4

In [25]:
# Question 10
import requests
from datetime import datetime, timedelta
import time
import random

def scrap_multiple_pages(start_date, end_date, max_page):
    page = 1
    previous_data = None  # Pour stocker les donnÃ©es de la page prÃ©cÃ©dente
    
    while page <= max_page:
        print(f"Collecte de la page {page}...")
        
        # RÃ©cupÃ©rer les donnÃ©es pour la page courante
        current_data = scrap_one_page(page, start_date, end_date)
        
        # Sauvegarder les donnÃ©es si la page n'est pas vide
        if current_data and 'events' in current_data and len(current_data['events']) > 0:
            save_json(current_data, page)
            
            # Si previous_data n'est pas None et la page actuelle est identique Ã  la page prÃ©cÃ©dente, arrÃªter la collecte
            if previous_data and 'events' in previous_data and current_data["events"][0] == previous_data["events"][0]:
                print("Les donnÃ©es de la page actuelle sont identiques Ã  la page prÃ©cÃ©dente. ArrÃªt de la collecte.")
                break
            
            # Mettre Ã  jour la page prÃ©cÃ©dente avec les donnÃ©es actuelles
            previous_data = current_data
        else:
            print("Page vide. ArrÃªt de la collecte.")
            break  # ArrÃªter la collecte si la page est vide
        
        # Attente alÃ©atoire pour Ã©viter d'Ãªtre bloquÃ© par l'API
        time.sleep(random.uniform(1, 3))
        
        page += 1

# Exemple d'appel de la fonction
scrap_multiple_pages("2024-10-01", "2024-10-31", 10)

Collecte de la page 1...
Collecte de la page 2...
Collecte de la page 3...
Collecte de la page 4...
Collecte de la page 5...
Collecte de la page 6...
Collecte de la page 7...
Collecte de la page 8...
Collecte de la page 9...
Collecte de la page 10...


In [26]:
# Question 11
def scrap_october_events():

    # ItÃ©rer sur chaque jour du mois d'octobre
    for day in range(1, 32):  # Du 1er au 31 octobre
        start_date = f"2024-10-{day:02d}"
        end_date = f"2024-10-{day+1:02d}" if day < 31 else "2024-11-01"  # GÃ©rer le dernier jour

        print(f"Collecte des Ã©vÃ©nements pour la pÃ©riode : {start_date} au {end_date}")

        # Appeler la fonction pour collecter les pages de la journÃ©e
        scrap_multiple_pages(start_date, end_date, max_page=1000)

        # Pause entre chaque journÃ©e pour Ã©viter de surcharger l'API
        time.sleep(random.uniform(1, 5))

# Exemple d'appel de la fonction principale
scrap_october_events()


Collecte des Ã©vÃ©nements pour la pÃ©riode : 2024-10-01 au 2024-10-02
Collecte de la page 1...
Collecte de la page 2...
Collecte de la page 3...
Collecte de la page 4...
Collecte de la page 5...
Collecte de la page 6...
Collecte de la page 7...
Collecte de la page 8...
Collecte de la page 9...
Collecte de la page 10...
Collecte de la page 11...
Collecte de la page 12...
Collecte de la page 13...
Collecte de la page 14...
Collecte de la page 15...
Collecte de la page 16...
Collecte de la page 17...
Collecte de la page 18...
Collecte de la page 19...
Collecte de la page 20...
Collecte de la page 21...
Collecte de la page 22...
Collecte de la page 23...
Collecte de la page 24...
Collecte de la page 25...
Collecte de la page 26...
Collecte de la page 27...
Collecte de la page 28...
Collecte de la page 29...
Collecte de la page 30...
Collecte de la page 31...
Collecte de la page 32...
Collecte de la page 33...
Collecte de la page 34...
Collecte de la page 35...
Collecte de la page 36...
Col

KeyboardInterrupt: 